In [81]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.wkb
from shapely.geometry import Point
from sqlalchemy import create_engine, text

In [83]:
from dotenv import load_dotenv

load_dotenv()

DBNAME = os.getenv('DBNAME')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')
CONNEXION = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}"

DBNAME_DVF_ANCIEN = os.getenv('DBNAME_DVF_ANCIEN')
DBNAME_ANCIEN = os.getenv('DBNAME_ANCIEN')
USER_ANCIEN = os.getenv('USER_ANCIEN')
PASSWORD_ANCIEN = os.getenv('PASSWORD_ANCIEN')
HOST_ANCIEN = os.getenv('HOST_ANCIEN')
PORT_ANCIEN = os.getenv('PORT_ANCIEN')
CONNEXION_ANCIEN = f"postgresql+psycopg2://{USER_ANCIEN}:{PASSWORD_ANCIEN}@{HOST_ANCIEN}:{PORT_ANCIEN}/{DBNAME_DVF_ANCIEN}"

PATH_DATA_G = os.getenv('PATH_DATA_G')
G_FILOSOFI = 'filosofi'
G_LOGEMENT = 'rp_logement'
G_DV3F = 'dv3f'

PATH_DATA_C = os.getenv('PATH_DATA_C')
C_CORRES = 'corres'

python-dotenv could not parse statement starting at line 16


### CORRESPONDANCE

In [48]:
engine = create_engine(CONNEXION)
requete_corres_iris_com = text("""
                               SELECT DISTINCT 
                               inseeiris::int as code_iris, 
                               inseecom::int as code_com 
                               from hab.iris_100
                    """)
requete_corres_com_epci = text("""
                               SELECT DISTINCT 
                               insee_comm::int as code_com, 
                               siren_epci::int as code_epci 
                               from hab.v_epcifp_comm
                    """)

with engine.connect() as connection:
    result = connection.execute(requete_corres_iris_com)
    corres_iris_com = pd.DataFrame(result.fetchall(), columns=result.keys())

    result = connection.execute(requete_corres_com_epci)
    corres_com_epci = pd.DataFrame(result.fetchall(), columns=result.keys())

corres_iris_com_epci = pd.merge(corres_iris_com, corres_com_epci, on='code_com', how='left')
corres_iris_com_epci

,code_iris,code_com,code_epci
0,341720803,34172,243400017
1,341570101,34157,200066355
2,341540106,34154,243400470
3,341450107,34145,243400520
4,341440000,34144,200042646
...,...,...,...
573,340250000,34025,243400769
574,341580000,34158,200066348
575,341160102,34116,243400017
576,341560000,34156,243400355


### CODE / ECHELLE

In [ ]:
# Création de l'engine SQLAlchemy
engine = create_engine(CONNEXION)

requete = text(
    """
select
    34 as code,
    'departement' as echelle
union
select 
    numero_siren::int as code,
    'epci' as echelle
from hab.v_epcifp_500 
union 
select 
    insee_com::int as code,
    'commune' as echelle
from hab.commune_500 
where insee_com like '34%' 
union
select 
    inseeiris::int as code,
    'iris' as echelle 
from hab.iris_100
order by code
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    code = pd.DataFrame(result.fetchall(), columns=result.keys())

code['echelle'].value_counts()

echelle
iris           578
commune        341
epci            17
departement      1
Name: count, dtype: int64

### ZONAGE GEOGRAPHQUE

In [4]:
# Création de l'engine SQLAlchemy
engine = create_engine(CONNEXION)

requete = text(
    """
    select *
    from hab.v_epcifp_500 
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    zonage_epci = pd.DataFrame(result.fetchall(), columns=result.keys())


requete = text(
    """
    select *
    from hab.commune_500 
    where insee_com like '34%' 
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    zonage_com = pd.DataFrame(result.fetchall(), columns=result.keys())


requete = text(
    """
    select *
    from hab.iris_100
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    rows = result.fetchall()
    columns = result.keys()
    dict_rows = [dict(zip(columns, row)) for row in rows]
    for row in dict_rows:
        row['shape'] = shapely.wkb.loads(row['shape'], hex=True)
    zonage_iris = gpd.GeoDataFrame(dict_rows, geometry='shape', crs="EPSG:2154")
    zonage_iris = zonage_iris.to_crs(epsg=4326) # WGS 84 -> Lambert 93
    zonage_iris['inseeiris'] = zonage_iris['inseeiris'].astype(int)


## DECRET

In [5]:
G_DECRET = 'decret'
chemin = os.path.join(PATH_DATA_G, G_DECRET, "2023_decret.xlsx")
decret = pd.read_excel(chemin, sheet_name="3")

## AIRDNA

In [6]:
G_AIRDNA = 'airdna'

In [7]:
print("Chargement des données Airbnb...")
chemin = os.path.join(PATH_DATA_G, G_AIRDNA, 'offre_airdna_34_2022.xlsx')
bnb2022 = pd.read_excel(chemin)
bnb2022 = bnb2022[bnb2022['Liste logement']=='Entire home/apt']
bnb2022 = bnb2022[['INSEE','Communes','Code EPCI','EPCI','Longitude','Latitude','Revenu annuel €',
                             'Nb jours réservés','Nb total jours de disponibilité']]
bnb2022.columns = ['code_com','com','code_epci','epci','longitude','latitude','rev_annuel',
                   'nb_jours_reserv','nb_jours_dispo']

chemin = os.path.join(PATH_DATA_G, G_AIRDNA, 'offre_airdna_34_2023.xlsx')
bnb2023 = pd.read_excel(chemin)
bnb2023 = bnb2023[bnb2023['Nature logement']=='Entire home/apt']
bnb2023 = bnb2023[['INSEE','Communes','Code EPCI','EPCI','Longitude','Latitude','Revenu annuel â‚¬',
                             'Nb jours rÃ©servÃ©s','Nb total jours de disponibilitÃ©']]
bnb2023.columns = ['code_com','com','code_epci','epci','longitude','latitude','rev_annuel',
                   'nb_jours_reserv','nb_jours_dispo']
print("Chargement des données Airbnb terminé.")

Chargement des données Airbnb...
Chargement des données Airbnb terminé.


In [ ]:
def traitement_bnb(bnb, code):
    """
    Fonction de traitement des données Airbnb
    """
    ### jointure IRIS
    bnb['geometry'] = bnb.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
    bnb_gdf = gpd.GeoDataFrame(bnb, geometry='geometry', crs="EPSG:4326")
    bnb_gdf = gpd.sjoin(bnb_gdf, zonage_iris, how="left", predicate="within")

    ### vérification de la jointure
    bnb_gdf["inseeiris_str"] = bnb_gdf["inseeiris"].astype(str)
    bnb_gdf["inseeiris_com"] = bnb_gdf["inseeiris_str"].str[:5]
    bnb_gdf["code_com_str"] = bnb_gdf["code_com"].astype(str)
    bnb_ss_iris_diff_iris = bnb_gdf[bnb_gdf["inseeiris_com"]==bnb_gdf["code_com_str"]]
    print(f"Nombre de logements Airbnb sans IRIS : {len(bnb_gdf) - len(bnb_ss_iris_diff_iris)}")

    ### aggrégation
    bnb_ss_iris_diff_iris['code_dep'] = 34
    bnb_ss_iris_diff_iris_agg = bnb_ss_iris_diff_iris.groupby([code]).agg(
        nombre_reserv_120=('nb_jours_reserv', lambda x: (x >= 120).sum()),
        nombre_dispo_120=('nb_jours_dispo', lambda x: (x >= 120).sum()),
        nombre_total=('nb_jours_reserv', 'count'),
        jours_reserves=('nb_jours_reserv', 'sum'),
        revenu=('rev_annuel', 'sum'),
        # jours_reserves_120=('nb_jours_reserv', lambda x: (bnb2023_ss_iris_diff_iris['nb_jours_dispo'] >= 120).sum()),
        jours_reserves_120=('nb_jours_reserv', lambda x: x[bnb_ss_iris_diff_iris.loc[x.index, 'nb_jours_dispo'] >= 120].sum()),
        # revenu_120=('rev_annuel', lambda x: (bnb2023_ss_iris_diff_iris['nb_jours_dispo'] >= 120).sum())
        revenu_120=('rev_annuel', lambda x: x[bnb_ss_iris_diff_iris.loc[x.index, 'nb_jours_dispo'] >= 120].sum())
    ).reset_index()
    bnb_ss_iris_diff_iris_agg['code'] = bnb_ss_iris_diff_iris_agg[code].astype(int)
    bnb_ss_iris_diff_iris_agg.drop(columns=[code], inplace=True)
    
    return bnb_ss_iris_diff_iris_agg

In [17]:
bnb_iris_2022 = traitement_bnb(bnb2022, 'inseeiris')
bnb_iris_2023 = traitement_bnb(bnb2023, 'inseeiris')

bnb_com_2022 = traitement_bnb(bnb2022, 'code_com')
bnb_com_2023 = traitement_bnb(bnb2023, 'code_com')

bnb_epci_2022 = traitement_bnb(bnb2022, 'code_epci')
bnb_epci_2023 = traitement_bnb(bnb2023, 'code_epci')

bnb_dep_2022 = traitement_bnb(bnb2022, 'code_dep')
bnb_dep_2023 = traitement_bnb(bnb2023, 'code_dep')

Nombre de logements Airbnb sans IRIS : 194
Nombre de logements Airbnb sans IRIS : 165
Nombre de logements Airbnb sans IRIS : 194
Nombre de logements Airbnb sans IRIS : 165
Nombre de logements Airbnb sans IRIS : 194
Nombre de logements Airbnb sans IRIS : 165
Nombre de logements Airbnb sans IRIS : 194
Nombre de logements Airbnb sans IRIS : 165


In [18]:
bnb_2022 = pd.concat([bnb_dep_2022, bnb_epci_2022, bnb_com_2022, bnb_iris_2022], axis=0).drop_duplicates().reset_index(drop=True)
bnb_2023 = pd.concat([bnb_dep_2023, bnb_epci_2023, bnb_com_2023, bnb_iris_2023], axis=0).drop_duplicates().reset_index(drop=True)

In [19]:
bnb_final = pd.merge(bnb_2022, bnb_2023, how='outer', on='code', suffixes=('_2022', '_2023'))

In [24]:
bnb_final_final = pd.merge(code, bnb_final, how='left', on='code')

In [25]:
# Calculer les évolutions et les écarts
bnb_final_final_final = bnb_final_final.assign(
    evol_bnb_120=(bnb_final_final['nombre_dispo_120_2023'] - bnb_final_final['nombre_dispo_120_2022']) / bnb_final_final['nombre_dispo_120_2022'],
    ecart_bnb_120=bnb_final_final['nombre_dispo_120_2023'] - bnb_final_final['nombre_dispo_120_2022'],
    ecart_bnb_tot=bnb_final_final['nombre_total_2023'] - bnb_final_final['nombre_total_2022'],
    evol_bnb_tot=(bnb_final_final['nombre_total_2023'] - bnb_final_final['nombre_total_2022']) / bnb_final_final['nombre_total_2022'],
    ecart_jours=bnb_final_final['jours_reserves_2023'] - bnb_final_final['jours_reserves_2022'],
    ecart_revenu=bnb_final_final['revenu_2023'] - bnb_final_final['revenu_2022'],
    ecart_revenu_jours=(bnb_final_final['revenu_2023'] / bnb_final_final['jours_reserves_2023']) - (bnb_final_final['revenu_2022'] / bnb_final_final['jours_reserves_2022'])
)

In [26]:
bnb_final_final_final

,code,echelle,nombre_reserv_120_2022,nombre_dispo_120_2022,nombre_total_2022,jours_reserves_2022,revenu_2022,jours_reserves_120_2022,revenu_120_2022,nombre_reserv_120_2023,...,revenu_2023,jours_reserves_120_2023,revenu_120_2023,evol_bnb_120,ecart_bnb_120,ecart_bnb_tot,evol_bnb_tot,ecart_jours,ecart_revenu,ecart_revenu_jours
0,34,departement,6233.0,15643.0,36444.0,2409136.0,290781081.0,1832776.0,215232204.0,5689.0,...,315493647.0,1829106.0,233359474.0,0.075881,1187.0,5407.0,0.148365,9670.0,24712566.0,9.734308
1,34001,commune,2.0,4.0,16.0,690.0,74625.0,455.0,48603.0,3.0,...,120060.0,793.0,81313.0,0.750000,3.0,3.0,0.187500,374.0,45435.0,4.686172
2,34002,commune,2.0,10.0,15.0,1028.0,177214.0,917.0,136338.0,3.0,...,228402.0,935.0,217818.0,0.100000,1.0,0.0,0.000000,-29.0,51188.0,56.243471
3,34003,commune,730.0,2663.0,5681.0,349231.0,39528819.0,264730.0,29208459.0,729.0,...,47218341.0,277452.0,33910387.0,0.079609,212.0,1228.0,0.216159,31245.0,7689522.0,10.915163
4,34004,commune,2.0,7.0,24.0,1457.0,885346.0,873.0,548588.0,10.0,...,1617070.0,2034.0,1459826.0,1.571429,11.0,4.0,0.166667,961.0,731724.0,61.113475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,343420000,iris,0.0,3.0,9.0,273.0,41789.0,172.0,23997.0,0.0,...,110938.0,340.0,89428.0,0.666667,2.0,1.0,0.111111,146.0,69149.0,111.695236
933,343430000,iris,0.0,1.0,20.0,683.0,105182.0,84.0,14349.0,0.0,...,124492.0,387.0,45836.0,5.000000,5.0,0.0,0.000000,99.0,19310.0,5.196931
934,343440101,iris,152.0,347.0,706.0,52884.0,4553978.0,41927.0,3639394.0,129.0,...,5062994.0,43516.0,3982483.0,0.167147,58.0,219.0,0.310198,1738.0,509016.0,6.578894
935,343440102,iris,280.0,650.0,1220.0,98197.0,8813928.0,78652.0,7049797.0,242.0,...,9882497.0,77856.0,7732345.0,0.055385,36.0,271.0,0.222131,471.0,1068569.0,10.401479


In [27]:
bnb_final_final_final['echelle'].value_counts()

echelle
iris           578
commune        341
epci            17
departement      1
Name: count, dtype: int64

## FILOSOFI

In [ ]:
chemin = os.path.join(PATH_DATA_G, G_FILOSOFI, 'filosofi_com_2020.xlsx')
filosofi_com_2020 = pd.read_excel(chemin, sheet_name=0)
chemin = os.path.join(PATH_DATA_G, G_FILOSOFI, 'filosofi_epci_2020.xlsx')
filosofi_epci_2020 = pd.read_excel(chemin)


filosofi = pd.concat([filosofi_com_2020, filosofi_epci_2020], ignore_index=True)

table = pd.merge(code, filosofi, how='left', left_on="code", right_on="CODGEO")
table = table.drop(columns=['CODGEO'])

## INSEE logement

In [ ]:
chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, 'base-ic-logement-2020 - Copie.xlsx')
ic_logement_2020 = pd.read_excel(chemin, sheet_name=0)
ic_logement_2020 = ic_logement_2020[['IRIS', 'P20_LOG', 'P20_RP', 'P20_RSECOCC', 'P20_LOGVAC', 'P20_RP_LOC', 'P20_RP_LOCHLMV']]

chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, 'base-ic-logement-2014 - Copie.xls')
ic_logement_2014 = pd.read_excel(chemin, sheet_name=0)
ic_logement_2014 = ic_logement_2014[['IRIS', 'P14_LOG', 'P14_RP', 'P14_RSECOCC', 'P14_LOGVAC', 'P14_RP_LOC', 'P14_RP_LOCHLMV']]

ic_insee = pd.merge(ic_logement_2020, ic_logement_2014, on=['IRIS'])
ic_insee.rename(columns={"IRIS": "code"}, inplace=True)


In [9]:
chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, "base-cc-logement - Copie.xlsx")

cc_logement_2020 = pd.read_excel(chemin, sheet_name=0)
cc_logement_2020 = cc_logement_2020[['CODGEO', 'P20_LOG', 'P20_RP', 'P20_RSECOCC', 'P20_LOGVAC', 'P20_RP_LOC', 'P20_RP_LOCHLMV']]

cc_logement_2014 = pd.read_excel(chemin, sheet_name=1)
cc_logement_2014 = cc_logement_2014[['CODGEO', 'P14_LOG', 'P14_RP', 'P14_RSECOCC', 'P14_LOGVAC', 'P14_RP_LOC', 'P14_RP_LOCHLMV']]

cc_insee = pd.merge(cc_logement_2020, cc_logement_2014, on="CODGEO")
cc_insee.rename(columns={'CODGEO': 'code'}, inplace=True)

In [11]:
chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, "epci_log_plus_2020.xlsx")
ep_logement_2020 = pd.read_excel(chemin)
chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, "epci_log_plus_2014.xlsx")
ep_logement_2014 = pd.read_excel(chemin)

# Rename the columns
ep_logement_2020.columns = ["INSEE", "P20_LOG", "P20_RP", "P20_RSECOCC", "P20_LOGVAC", "P20_RP_LOC"]
ep_logement_2014.columns = ["INSEE", "nom", "P14_RP_LOC", "P14_RSECOCC", "P14_LOGVAC", "P14_LOG", "P14_RP"]

# Select the desired columns
ep_logement_2020 = ep_logement_2020[["INSEE", "P20_LOG", "P20_RP", "P20_RSECOCC", "P20_LOGVAC", "P20_RP_LOC"]]
ep_logement_2014 = ep_logement_2014[["INSEE", "P14_LOG", "P14_RP", "P14_RSECOCC", "P14_LOGVAC", "P14_RP_LOC"]]

# Merge the dataframes
ep_insee = pd.merge(ep_logement_2020, ep_logement_2014, on="INSEE")
ep_insee.rename(columns={"INSEE": "code"}, inplace=True)
# table_epci = pd.merge(table_epci, ep_insee, left_on="code_epci", right_on="INSEE")

In [12]:
rp_logement = pd.concat([ic_insee, cc_insee, ep_insee], ignore_index=True)
rp_logement

,code,P20_LOG,P20_RP,P20_RSECOCC,P20_LOGVAC,P20_RP_LOC,P20_RP_LOCHLMV,P14_LOG,P14_RP,P14_RSECOCC,P14_LOGVAC,P14_RP_LOC,P14_RP_LOCHLMV
0,340010000,892.988099,748.530568,72.733862,71.723669,153.593122,16.115962,820.000000,655.000000,78.000000,87.000000,139.000000,21.000000
1,340020000,681.019051,541.674059,53.837838,85.507154,128.908334,10.501239,585.887378,450.676664,44.382143,90.828571,105.919400,4.153702
2,340030101,2306.016121,1292.183201,644.191934,369.640986,908.579168,14.325485,2163.467504,1111.132298,488.327036,564.008170,744.963673,8.998470
3,340030102,1831.016697,1338.315011,275.123113,217.578574,737.591664,73.244900,1770.512804,1301.840927,217.786469,250.885408,712.586611,3.859058
4,340030103,3342.035187,2820.364045,310.451882,211.219260,1699.735605,305.184346,2968.523742,2413.825911,224.571320,330.126511,1417.202588,128.883287
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,248200057,11519.000000,9399.000000,877.000000,1243.000000,2556.000000,NaN,11017.000000,8901.000000,816.000000,1300.000000,2394.000000,NaN
1080,248200065,5331.000000,4234.000000,416.000000,680.000000,933.000000,NaN,5149.000000,4067.000000,416.000000,665.000000,823.000000,NaN
1081,248200099,40093.000000,35345.000000,652.000000,4097.000000,10577.000000,NaN,37554.000000,33442.000000,414.000000,3697.000000,10147.000000,NaN
1082,248200107,6606.000000,3780.000000,2128.000000,698.000000,746.000000,NaN,6507.000000,3613.000000,2028.000000,865.000000,663.000000,NaN


Calcul des variables supplémentaires
table_iris['point_RP'] = (table_iris['P20_RP'] / table_iris['P20_LOG']) - (table_iris['P14_RP'] / table_iris['P14_LOG'])
table_iris['deriv_RP'] = (table_iris['P20_RP'] - table_iris['P14_RP']) / table_iris['P14_RP']
table_iris['croissance_relative_RP_log'] = ((table_iris['P20_RP'] - table_iris['P14_RP']) / table_iris['P14_RP']) - ((table_iris['P20_LOG'] - table_iris['P14_LOG']) / table_iris['P14_LOG'])
table_iris['gain_plpr'] = (table_iris['P20_RP_LOC'] - table_iris['P20_RP_LOCHLMV']) - (table_iris['P14_RP_LOC'] - table_iris['P14_RP_LOCHLMV'])
table_iris['taux_d_implantation'] = table_iris['nombre_dispo_120_2023'] / (table_iris['P20_LOG'] - table_iris['P20_RP'])
table_iris['taux_d_implantation_parc_tot'] = table_iris['nombre_total_2023'] / table_iris['P20_LOG']
table_iris['taux_d_implantation_parc_tot_120'] = table_iris['nombre_dispo_120_2023'] / table_iris['P20_LOG']
table_iris['potentiel_reconquete'] = table_iris['nombre_dispo_120_2023']
table_iris['taux_reconquete'] = table_iris['nombre_dispo_120_2023'] / (table_iris['P20_RP_LOC'] - table_iris['P14_RP_LOC'])
table_iris['deriv_plpr'] = ((table_iris['P20_RP_LOC'] - table_iris['P20_RP_LOCHLMV']) - (table_iris['P14_RP_LOC'] - table_iris['P14_RP_LOCHLMV'])) / (table_iris['P14_RP_LOC'] - table_iris['P14_RP_LOCHLMV'])
table_iris['taux_bnb_parc_total'] = table_iris['nombre_dispo_120_2023'] / table_iris['P20_LOG']
table_iris['part_rs'] = table_iris['P20_RSECOCC'] / table_iris['P20_LOG']
table_iris['part_vac'] = table_iris['P20_LOGVAC'] / table_iris['P20_LOG']
table_iris['deriv_rs'] = (table_iris['P20_RSECOCC'] - table_iris['P14_RSECOCC']) / table_iris['P14_RSECOCC']
table_iris['deriv_vac'] = (table_iris['P20_LOGVAC'] - table_iris['P14_LOGVAC']) / table_iris['P14_LOGVAC']
table_iris['bnb_120_sur_log_tot'] = table_iris['nombre_dispo_120_2023'] / table_iris['P20_LOG']

## DV3F

### agrege

In [80]:
def read_and_filter(filepath, annee, type='maison', echelle=False):
    """
    Variables:
        filepath: CHEMIN du fichier
        type: 'appart' ou 'maison'
        dep (valeur par défault False):
            True pour département (Hérault)
            False pour autre territoires de l'Hérault
    Entrée: CHEMIN vers un fichier
    Sortie:
        if dep=False:
            pd.DataFrame avec ['codgeo', f'prix_median_{type}_{annee}'] comme colonnes
        else:
            int, prix médian au département de l'hérault
    """
    if type == 'maison':
        sheet_index = 2
        col_select = 'valeurfonc_median_cod111'
    elif type == 'appart':
        sheet_index = 5
        col_select = 'valeurfonc_median_cod121'
    df = pd.read_excel(filepath, sheet_name=sheet_index)
    if echelle=='dep':
        df = df[df['dep'] == '34'][col_select].values[0]
    elif echelle=='epci':
        epci_34 = ['200071058', '243400819', '200066348', '243400736', '200042646',
                   '243400769', '243400694', '243400355', '200022986', '200042653',
                   '243400017', '200066355', '243400520', '243400470', '243400488',
                   '200017341', '200066553']
        epci_34_df = pd.DataFrame(epci_34, columns=['code_epci'])
        df = pd.merge(df, epci_34_df, left_on="epci", right_on="code_epci")
        df = df[["epci", col_select]]
        df.columns = ['codgeo', f'prix_median_{type}_{annee}']
    else:
        df = df[df['codgeo'].str.startswith('34')]
        df = df[['codgeo', col_select]].dropna()
        df.columns = ['codgeo', f'prix_median_{type}_{annee}']
    return df

#### epci

In [ ]:
# Partie 4: DV3F ----
# https://cerema.app.box.com/v/dv3f-indicateurs?page=7
prix_median_maison_34_2020_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_departements_2020_2022.xlsx", "2020_2022", "maison", 'dep')
prix_median_appart_34_2020_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_departements_2020_2022.xlsx", "2020_2022", "appart", 'dep')

prix_median_maison_34_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_departements_2020_2022.xlsx", "2022", "maison", 'dep')
prix_median_appart_34_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_departements_2020_2022.xlsx", "2022", "appart", 'dep')

dv3f_maison_2020_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2020_2022.xlsx", "2020_2022", "maison", 'epci')

dv3f_appart_2020_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2020_2022.xlsx", "2020_2022", "appart", 'epci')

dv3f_maison_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2020_2022.xlsx", "2022", "maison", 'epci')

dv3f_appart_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2020_2022.xlsx", "2022", "appart", 'epci')

dv3f_maison_2020 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2020.xlsx", "2020", "maison", 'epci')

dv3f_appart_2020 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2020.xlsx", "2020", "appart", 'epci')

dv3f_maison_2014 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2014.xlsx", "2014", "maison", 'epci')

dv3f_appart_2014 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2014.xlsx", "2014", "appart", 'epci')

dv3f_maison = pd.merge(dv3f_maison_2014, dv3f_maison_2020, on="codgeo")
dv3f_maison["croissance_prix_maison"] = (dv3f_maison["prix_median_maison_2020"] - dv3f_maison["prix_median_maison_2014"]) / dv3f_maison["prix_median_maison_2014"]

dv3f_appart = pd.merge(dv3f_appart_2014, dv3f_appart_2020, on="codgeo")
dv3f_appart["croissance_prix_appart"] = (dv3f_appart["prix_median_appart_2020"] - dv3f_appart["prix_median_appart_2014"]) / dv3f_appart["prix_median_appart_2014"]

dv3f_maison_2019_2021 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2019_2021.xlsx", "2019_2021", "maison", 'epci')

dv3f_appart_2019_2021 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2019_2021.xlsx", "2019_2021", "appart", 'epci')

dv3f_maison_2013_2015 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2013_2015.xlsx", "2013_2015", "maison", 'epci')

dv3f_appart_2013_2015 = read_and_filter("données/dv3f/dv3f_indicateurs_epci_2013_2015.xlsx", "2013_2015", "appart", 'epci')

dv3f_maison_pas = pd.merge(dv3f_maison_2013_2015, dv3f_maison_2019_2021, on="codgeo")
dv3f_maison_pas["croissance_prix_maison_pas"] = (dv3f_maison_pas["prix_median_maison_2019_2021"] - dv3f_maison_pas["prix_median_maison_2013_2015"]) / dv3f_maison_pas["prix_median_maison_2013_2015"]

dv3f_appart_pas = pd.merge(dv3f_appart_2013_2015, dv3f_appart_2019_2021, on="codgeo")
dv3f_appart_pas["croissance_prix_appart_pas"] = (dv3f_appart_pas["prix_median_appart_2019_2021"] - dv3f_appart_pas["prix_median_appart_2013_2015"]) / dv3f_appart_pas["prix_median_appart_2013_2015"]

dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_pas, on="codgeo", suffixes=("", "_pas"))
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_2022, on="codgeo", suffixes=("", "_2022"))
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_2020_2022, on="codgeo", suffixes=("", "_2020_2022"))
dv3f_maison["prix_median_maison_2022_norm"] = dv3f_maison["prix_median_maison_2022"] / prix_median_maison_34_2022
dv3f_maison["prix_median_maison_2020_2022_norm"] = dv3f_maison["prix_median_maison_2020_2022"] / prix_median_maison_34_2020_2022

dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_pas, on="codgeo", suffixes=("", "_pas"))
dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_2022, on="codgeo", suffixes=("", "_2022"))
dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_2020_2022, on="codgeo", suffixes=("", "_2020_2022"))
dv3f_appart["prix_median_appart_2022_norm"] = dv3f_appart["prix_median_appart_2022"] / prix_median_appart_34_2022
dv3f_appart["prix_median_appart_2020_2022_norm"] = dv3f_appart["prix_median_appart_2020_2022"] / prix_median_appart_34_2020_2022

dv3f_prix_median = pd.merge(dv3f_maison, dv3f_appart, on="codgeo")

# Calculate the prix_median_2020_2022 column
dv3f_prix_median['prix_median_2020_2022'] = np.where(
    np.isnan(dv3f_prix_median['prix_median_appart_2020_2022']),
    dv3f_prix_median['prix_median_maison_2020_2022'] / prix_median_maison_34_2020_2022,
    (dv3f_prix_median['prix_median_maison_2020_2022'] + dv3f_prix_median['prix_median_appart_2020_2022']) /
    (prix_median_maison_34_2020_2022 + prix_median_appart_34_2020_2022)
)

# Calculate the croissance_prix_pas column
dv3f_prix_median['croissance_prix_pas'] = np.where(
    np.isnan(dv3f_prix_median['croissance_prix_appart_pas']),
    dv3f_prix_median['croissance_prix_maison_pas'],
    (dv3f_prix_median['croissance_prix_maison_pas'] + dv3f_prix_median['croissance_prix_appart_pas']) / 2
)

# Calculate the croissance_prix column
dv3f_prix_median['croissance_prix'] = np.where(
    np.isnan(dv3f_prix_median['croissance_prix_appart']),
    dv3f_prix_median['croissance_prix_maison'],
    (dv3f_prix_median['croissance_prix_maison'] + dv3f_prix_median['croissance_prix_appart']) / 2
)

#### com

In [84]:
chemin = os.path.join(PATH_DATA_G, G_DV3F, "dv3f_prix_volumes_communes_2014.xlsx")
dv3f_maison_2014 = read_and_filter(chemin, 2014)

FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Esp-mission-Patrimoine\\documentLibrary\\Habitat_Amenagement\\OBSERVATOIRE\\02_ACTIVITES\\ETUDES\\ETUDES_ADIL\\TRAVAIL\\BNB\\CLUSTER\\data\\dv3f\\dv3f_prix_volumes_communes_2014.xlsx'

In [ ]:
print('Chargement des données DV3F...')
# Chemin du dossier de téléchargement, G:/ est un lecteur réseau ne recevant pas les fichiers lourds...
chemin = r"G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\03_AUTRES\ALTERNANT\ENZO\Avancement\Airbnb\données\dv3f"
download = r"C:\Users\ejaguin\Downloads"

dv3f_maison_2014 = read_and_filter(download+"/dv3f_prix_volumes_communes_2014.xlsx", 2014)
dv3f_maison_2020 = read_and_filter(download+"/dv3f_indicateurs_communes_2020.xlsx", 2020)
dv3f_maison_2022 = read_and_filter(download+"/dv3f_indicateurs_communes_2022.xlsx", 2022)
dv3f_maison_2013_2015 = read_and_filter(download+"/dv3f_indicateurs_communes_2013_2015.xlsx", '2013_2015')
dv3f_maison_2019_2021 = read_and_filter(download+"/dv3f_indicateurs_communes_2019_2021.xlsx", '2019_2021')
dv3f_maison_2020_2022 = read_and_filter(download+"/dv3f_indicateurs_communes_2020_2022.xlsx", '2020_2022')

dv3f_appart_2014 = read_and_filter(download+"/dv3f_indicateurs_communes_2014.xlsx", 2014, type='appart')
dv3f_appart_2020 = read_and_filter(download+"/dv3f_indicateurs_communes_2020.xlsx", 2020, type='appart')
dv3f_appart_2022 = read_and_filter(download+"/dv3f_indicateurs_communes_2022.xlsx", 2022, type='appart')
dv3f_appart_2013_2015 = read_and_filter(download+"/dv3f_indicateurs_communes_2013_2015.xlsx", '2013_2015', type='appart')
dv3f_appart_2019_2021 = read_and_filter(download+"/dv3f_indicateurs_communes_2019_2021.xlsx", '2019_2021', type='appart')
dv3f_appart_2020_2022 = read_and_filter(download+"/dv3f_indicateurs_communes_2020_2022.xlsx", '2020_2022', type='appart')


prix_median_maison_34_2022 = read_and_filter(chemin+"/dv3f_indicateurs_departements_2022.xlsx", 2022, echelle='dep')
prix_median_maison_34_2020_2022 = read_and_filter(chemin+"/dv3f_indicateurs_departements_2022.xlsx", '2020_2022', echelle='dep')

prix_median_appart_34_2022 = read_and_filter(chemin+"/dv3f_indicateurs_departements_2022.xlsx", 2022, 'appart', echelle='dep')
prix_median_appart_34_2020_2022 = read_and_filter(chemin+"/dv3f_indicateurs_departements_2022.xlsx", '2020_2022', 'appart', echelle='dep')

# prix_median_maison_34_2020_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_departements_2020_2022.xlsx", 2, col_select='valeurfonc_median_cod111', echelle='dep')
# prix_median_appart_34_2020_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_communes_2020_2022.xlsx", 5, col_select='valeurfonc_median_cod121', echelle='dep')

# prix_median_maison_34_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_communes_2022.xlsx", 2, 'codgeo', 'valeurfonc_median_cod111', 'prix_median_maison_2022')
# prix_median_appart_34_2022 = read_and_filter("données/dv3f/dv3f_indicateurs_communes_2022.xlsx", 5, 'codgeo', 'valeurfonc_median_cod121', 'prix_median_appart_2022')

# dv3f_maison_34_2020_2022 = pd.read_excel("données/dv3f/dv3f_indicateurs_departements_2020_2022.xlsx", sheet_name=2)
# prix_median_maison_34_2020_2022 = dv3f_maison_34_2020_2022[dv3f_maison_34_2020_2022[echelle='dep'] == '34']['valeurfonc_median_cod111'].values

# dv3f_appart_34_2020_2022 = pd.read_excel("données/dv3f/dv3f_indicateurs_departements_2020_2022.xlsx", sheet_name=5)
# prix_median_appart_34_2020_2022 = dv3f_appart_34_2020_2022[dv3f_appart_34_2020_2022[echelle='dep'] == '34']['valeurfonc_median_cod121'].values

# dv3f_maison_34_2022 = pd.read_excel("données/dv3f/dv3f_indicateurs_departements_2022.xlsx", sheet_name=2)
# prix_median_maison_34_2022 = dv3f_maison_34_2022[dv3f_maison_34_2022[echelle='dep'] == '34']['valeurfonc_median_cod111'].values

# dv3f_appart_34_2022 = pd.read_excel("données/dv3f/dv3f_indicateurs_departements_2022.xlsx", sheet_name=5)
# prix_median_appart_34_2022 = dv3f_appart_34_2022[dv3f_appart_34_2022[echelle='dep'] == '34']['valeurfonc_median_cod121'].values

# Fusion des données et calcul de la croissance des prix
dv3f_maison = pd.merge(dv3f_maison_2014, dv3f_maison_2020, on='codgeo')
dv3f_maison['croissance_prix_maison'] = (dv3f_maison['prix_median_maison_2020'] - dv3f_maison['prix_median_maison_2014']) / dv3f_maison['prix_median_maison_2014']

dv3f_appart = pd.merge(dv3f_appart_2014, dv3f_appart_2020, on='codgeo')
dv3f_appart['croissance_prix_appart'] = (dv3f_appart['prix_median_appart_2020'] - dv3f_appart['prix_median_appart_2014']) / dv3f_appart['prix_median_appart_2014']

dv3f_maison_pas = pd.merge(dv3f_maison_2013_2015, dv3f_maison_2019_2021, on='codgeo')
dv3f_maison_pas['croissance_prix_maison_pas'] = (dv3f_maison_pas['prix_median_maison_2019_2021'] - dv3f_maison_pas['prix_median_maison_2013_2015']) / dv3f_maison_pas['prix_median_maison_2013_2015']

dv3f_appart_pas = pd.merge(dv3f_appart_2013_2015, dv3f_appart_2019_2021, on='codgeo')
dv3f_appart_pas['croissance_prix_appart_pas'] = (dv3f_appart_pas['prix_median_appart_2019_2021'] - dv3f_appart_pas['prix_median_appart_2013_2015']) / dv3f_appart_pas['prix_median_appart_2013_2015']

# Fusion des données DV3F pour les maisons et les appartements avec les années 2022 et 2020-2022
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_pas, how='outer', suffixes=('', '_pas'), on='codgeo')
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_2022, how='outer', suffixes=('', '_2022'), on='codgeo')
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_2020_2022, how='outer', suffixes=('', '_2020_2022'), on='codgeo')
dv3f_maison['prix_median_maison_2022_norm'] = dv3f_maison['prix_median_maison_2022'] / prix_median_maison_34_2022
dv3f_maison['prix_median_maison_2020_2022_norm'] = dv3f_maison['prix_median_maison_2020_2022'] / prix_median_maison_34_2020_2022

dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_pas, how='outer', suffixes=('', '_pas'), on='codgeo')
dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_2022, how='outer', suffixes=('', '_2022'), on='codgeo')
dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_2020_2022, how='outer', suffixes=('', '_2020_2022'), on='codgeo')
dv3f_appart['prix_median_appart_2022_norm'] = dv3f_appart['prix_median_appart_2022'] / prix_median_appart_34_2022
dv3f_appart['prix_median_appart_2020_2022_norm'] = dv3f_appart['prix_median_appart_2020_2022'] / prix_median_appart_34_2020_2022

# Fusion des données pour les maisons et les appartements
dv3f_prix_median = pd.merge(dv3f_maison, dv3f_appart, how='outer', on='codgeo')

# Calcul des colonnes prix_median_2020_2022, croissance_prix_pas et croissance_prix
dv3f_prix_median['prix_median_2020_2022'] = np.where(
pd.isna(dv3f_prix_median['prix_median_appart_2020_2022']),
dv3f_prix_median['prix_median_maison_2020_2022'] / prix_median_maison_34_2020_2022,
(dv3f_prix_median['prix_median_maison_2020_2022'] + dv3f_prix_median['prix_median_appart_2020_2022']) / (prix_median_maison_34_2020_2022 + prix_median_appart_34_2020_2022)
)


dv3f_prix_median['croissance_prix_pas'] = dv3f_prix_median.apply(lambda row: row['croissance_prix_maison_pas'] if pd.isna(row['croissance_prix_appart_pas']) else (row['croissance_prix_maison_pas'] + row['croissance_prix_appart_pas']) / 2, axis=1)

dv3f_prix_median['croissance_prix'] = dv3f_prix_median.apply(lambda row: row['croissance_prix_maison'] if pd.isna(row['croissance_prix_appart']) else (row['croissance_prix_maison'] + row['croissance_prix_appart']) / 2, axis=1)

### détail

In [ ]:
chemin = os.path.join(PATH_DATA_C, C_CORRES, "corres_parcelle_iris.csv")
corres_parcelle_iris = pd.read_csv(chemin, sep='\t', header=0, dtype=str)
corres_parcelle_iris['code_iris'] = corres_parcelle_iris['code_iris'].astype('Int64')

def traitement_dvf(df, year, code):  
    df['id_par'] = df['l_idpar'].apply(lambda x: x[0]) # Prends la première parcelle. A voir si on peut faire mieux.
    df = df.merge(corres_parcelle_iris, on="id_par", how='left')
    df = df.merge(corres_iris_com_epci, on='code_iris')
    # calcul pour chaque iris et chaque codtypbien, le nombre et la médiane du prix de vente
    iris_codtype_compte_med = df.groupby([code, 'codtypbien']).agg(
            compte=('valeurfonc', 'count'),
            médiane=('valeurfonc', 'median')
        ).reset_index()
    # calcul pour chaque codtypbien, la médiane du prix de vente
    total_mediane_codtype = df.groupby('codtypbien')['valeurfonc'].median().reset_index().rename(columns={'valeurfonc': 'med_typbien'})
    # calcul pour chaque iris, le nombre total de prix de vente
    total_compte = iris_codtype_compte_med.groupby(code)['compte'].sum().reset_index().rename(columns={'compte': 'total_compte'})
    # fusion des 3 dataframes
    iris_codtype_compte_med_medtypbien = iris_codtype_compte_med.merge(total_mediane_codtype, on="codtypbien", how='left')
    iris_codtype_compte_med_medtypbien_totcompte = iris_codtype_compte_med_medtypbien.merge(total_compte, on=code, how='left')
    data = iris_codtype_compte_med_medtypbien_totcompte
    # calcul de l'indicateur
    data['indicateur'] = (data['compte'] / data['total_compte']) * (data['médiane'] / data['med_typbien'])
    # calcul de l'indicateur par iris
    ind_marche_iris = data[[code, 'indicateur']].groupby(code).sum().reset_index()

    return ind_marche_iris.rename(columns={code:'code', 'indicateur': f'indicateur_marche_{year}'})

In [ ]:
engine_ancien = create_engine(CONNEXION_ANCIEN)

codtypbien_list = ('1112', '1113', '12121', '12122', '12125', '12123', '12124', '12131', '12132', '12135', '12133', '12134')
requete_type = f"""SELECT idmutation, anneemut, libtypbien, codtypbien, valeurfonc, sbati, l_idpar
    FROM dvf.mutation 
    WHERE codtypbien IN {codtypbien_list}
    """

requetes = {
    "2022": text(f"""
    {requete_type} 
    AND anneemut IN (2020, 2021, 2022)
    """),
    "2020": text(f"""
    {requete_type} 
    AND anneemut IN (2019, 2020, 2021)
    """),
    "2014": text(f"""
    {requete_type} 
    AND anneemut IN (2013, 2014, 2015)
    """)
}

with engine_ancien.connect() as connection:
    result_2022 = connection.execute(requetes['2022'])
    dvf_2022 = pd.DataFrame(result_2022.fetchall(), columns=result_2022.keys())

    result_2020 = connection.execute(requetes['2020'])
    dvf_2020 = pd.DataFrame(result_2020.fetchall(), columns=result_2020.keys())

    result_2014 = connection.execute(requetes['2014'])
    dvf_2014 = pd.DataFrame(result_2014.fetchall(), columns=result_2014.keys())



In [78]:
indicateur_marche_iris_2022 = traitement_dvf(dvf_2022, 2022, 'code_iris')
indicateur_marche_iris_2020 = traitement_dvf(dvf_2020, 2020, 'code_iris')
indicateur_marche_iris_2014 = traitement_dvf(dvf_2014, 2014, 'code_iris')

indicateur_marche_com_2022 = traitement_dvf(dvf_2022, 2022, 'code_com')
indicateur_marche_com_2020 = traitement_dvf(dvf_2020, 2020, 'code_com')
indicateur_marche_com_2014 = traitement_dvf(dvf_2014, 2014, 'code_com')

indicateur_marche_epci_2022 = traitement_dvf(dvf_2022, 2022, 'code_epci')
indicateur_marche_epci_2020 = traitement_dvf(dvf_2020, 2020, 'code_epci')
indicateur_marche_epci_2014 = traitement_dvf(dvf_2014, 2014, 'code_epci')

indicateur_marche_2022 = pd.concat([indicateur_marche_iris_2022, indicateur_marche_com_2022, indicateur_marche_epci_2022], axis=0)
indicateur_marche_2020 = pd.concat([indicateur_marche_iris_2020, indicateur_marche_com_2020, indicateur_marche_epci_2020], axis=0)
indicateur_marche_2014 = pd.concat([indicateur_marche_iris_2014, indicateur_marche_com_2014, indicateur_marche_epci_2014], axis=0)

indicateur_marche = indicateur_marche_2022.merge(indicateur_marche_2020, on="code", how='outer', suffixes=("", "_2020"))
indicateur_marche = indicateur_marche.merge(indicateur_marche_2014, on="code", how='outer', suffixes=("", "_2014"))
indicateur_marche["evol_indicateur_marche"] = indicateur_marche["indicateur_marche_2020"] - indicateur_marche["indicateur_marche_2014"]

In [79]:
indicateur_marche

,code,indicateur_marche_2022,indicateur_marche_2020,indicateur_marche_2014,evol_indicateur_marche
0,34001,0.787574,0.660959,0.76283,-0.101871
1,34002,0.71949,0.746362,0.511923,0.234439
2,34003,0.837231,0.834856,0.898106,-0.06325
3,34004,0.322207,0.310011,0.582927,-0.272916
4,34005,1.232033,1.382863,1.030488,0.352375
...,...,...,...,...,...
748,343420000,1.774741,1.432021,1.595122,-0.163101
749,343430000,1.055607,1.175987,1.232609,-0.056622
750,343440101,1.308202,1.300227,1.285436,0.014791
751,343440102,1.355452,1.338558,1.388897,-0.050339
